<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/heejinohn/combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wrds datetime
import pandas as pd, numpy as np, wrds
import datetime as dt

     |████████████████████████████████| 61kB 3.9MB/s 
     |████████████████████████████████| 3.0MB 10.9MB/s 
     |████████████████████████████████| 256kB 38.1MB/s 


# Read datasets

In [ ]:
option_list = pd.read_sas("/content/drive/MyDrive/option_ticker.sas7bdat",encoding="utf-8")

In [ ]:
db = wrds.Connection()

Enter your WRDS username [root]:hohn
Enter your password:··········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [ ]:
om_name = (db.get_table('optionm','secnmd')
                .groupby(['secid','ticker','issuer'])['effect_date']
                .agg(['min','max'])
                .reset_index().rename({'min':'start','max':'end'},axis=1))

In [ ]:
om_name['ticker'] = np.where(om_name['ticker']=='ZZZZ',np.NaN,om_name['ticker'])

Unique ticker list from OptionMetrics

In [ ]:
ticker = list(om_name.ticker.dropna().unique())

---
# *Don't run here if file already saved.*
---

## Grab Reddit submissions using Pushshift API



In [ ]:
api = PushshiftAPI()

We only examine posts up to 2020.

In [ ]:
if not os.path.exists('/content/drive/MyDrive/submission.parquet.gzip'):
    field = ['id','created','created_utc','num_comments','score',
            'upvote_ratio','full_link','title','selftext','link_flair_text']
    end_epoch = int(dt.datetime(2021,1,1).timestamp())
    gen = api.search_submissions(subreddit='wallstreetbets',
                                before=end_epoch)
    submission = pd.DataFrame([post.d_ for post in gen])[field]
    submission['date'] = pd.to_datetime(submission['created'], unit='s', utc=True)
    submission['date_utc'] = pd.to_datetime(submission['created_utc'], unit='s', utc=True)
    col = submission.columns.to_list()
    col = col[0:1] + col[-2:] + col[3:-2]
    submission = submission[col]
    #submission = pd.DataFrame([post.d_ for post in gen])
    submission.to_parquet('/content/drive/MyDrive/submission.parquet.gzip', compression='gzip')

---

In [ ]:
submission = pd.read_parquet('/content/drive/MyDrive/submission.parquet.gzip')

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.56 GB -- Worker memory limit: 2.00 GB


In [ ]:
submission.set_index(['id'],inplace=True)

# Link ticker to WSB posts

In [ ]:
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client

Client Scheduler: inproc://172.28.0.2/952/1 Dashboard: http://172.28.0.2:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import dask
import dask.array as da
import dask.dataframe as dd
from dask.diagnostics import ProgressBar 

In [ ]:
submission.shape

(563551, 9)

In [ ]:
sub_dask = dd.from_pandas(submission, npartitions=1000)

In [ ]:
b_set = set('$' + tic for tic in ticker)
c_set = set(ticker)
sub_dask['title_split'] = sub_dask['title'].str.split()
sub_dask['selftext_split'] = sub_dask['selftext'].str.split()

In [ ]:
sub_dask['title_ticker'] = sub_dask.title.map_partitions(find_ticker)
sub_dask['title_ticker'] = sub_dask.selftext.map_partitions(find_ticker)

ValueError: ignored

In [ ]:
submission.to_parquet('/content/drive/MyDrive/submission_ticker.parquet.gzip', compression='gzip')

Additional corrections

* Intersect title and body
* If intersect with $, stop
* Use comments

Variables to capture from posts

* Number of posts
* Combined rewards
* Comments
* Score